# Wind shear estimation

* RESKit will vertically adjust wind speeds to accoutn for wind shear
* Uses a logarithmic wind profile approach
* Requires estimating roughness length, and having a "known" wind speed

In [1]:
import reskit as rk

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Create a weather source, and load data
src = rk.weather.MerraSource(rk.TEST_DATA["merra-like"], bounds=[5,49,7,52], verbose=False)
src.sload_elevated_wind_speed()

# Get an adjusted time series for a specific location

In [3]:
# Extract raw wind speed and GHI data
location = (6.0,50.5)

windspeeds = src.get("elevated_wind_speed", locations=location)
print(windspeeds.head())

2015-01-01 00:30:00+00:00    5.457981
2015-01-01 01:30:00+00:00    5.324582
2015-01-01 02:30:00+00:00    5.161159
2015-01-01 03:30:00+00:00    5.035860
2015-01-01 04:30:00+00:00    5.254663
Name: (6.00000,50.50000), dtype: float32


In [4]:
# Estimate roughness length at the location from a land-cover dataset
#  - In this example, Corine Land Cover will be used
#  - in this case, roughness lengths are taken from the suggestions by Silva et al.
#    "Roughness Length Classification Of Corine Land Cover Classes". 2007

roughness = rk.wind.roughness_from_clc(
    clc_path=rk.TEST_DATA['clc-aachen_clipped.tif'],
    loc=location)

print("Estimated roughness is {}m".format(roughness) )

Estimated roughness is 0.75m


In [5]:
# Apply log-law projection
projected_windspeed = rk.wind.apply_logarithmic_profile_projection(
    measured_wind_speed=windspeeds,
    measured_height=50, # The MERRA dataset offers windspeeds at 50m
    target_height=120,  # Assuming we want to project up to 120m
    roughness=roughness)

print(projected_windspeed.head())

2015-01-01 00:30:00+00:00    6.595749
2015-01-01 01:30:00+00:00    6.434542
2015-01-01 02:30:00+00:00    6.237052
2015-01-01 03:30:00+00:00    6.085633
2015-01-01 04:30:00+00:00    6.350049
Name: (6.00000,50.50000), dtype: float32


# Get adjusted time series for multiple locations at once

In [6]:
# Extract raw wind speed and GHI data
locations = [(6.25,51.), (6.50,51.), (6.25,50.75)]

windspeeds = src.get("elevated_wind_speed", locations=locations)
windspeeds.head()

,"(6.25000,51.00000)","(6.50000,51.00000)","(6.25000,50.75000)"
2015-01-01 00:30:00+00:00,6.117667,6.117667,6.117667
2015-01-01 01:30:00+00:00,6.147689,6.147689,6.147689
2015-01-01 02:30:00+00:00,6.101908,6.101908,6.101908
2015-01-01 03:30:00+00:00,5.905268,5.905268,5.905268
2015-01-01 04:30:00+00:00,5.990820,5.990820,5.990820


In [7]:
roughness = rk.wind.roughness_from_clc(
    clc_path=rk.TEST_DATA['clc-aachen_clipped.tif'],
    loc=locations, )

print("Estimated roughnesses are:")
print("  At {}: {}m".format(str(locations[0]), roughness[0]) )
print("  At {}: {}m".format(str(locations[1]), roughness[1]) )
print("  At {}: {}m".format(str(locations[2]), roughness[2]) )


Estimated roughnesses are:
  At (6.25, 51.0): 0.5m
  At (6.5, 51.0): 0.05m
  At (6.25, 50.75): 0.03m


In [8]:
projected_windspeed = rk.wind.apply_logarithmic_profile_projection(
    measured_wind_speed=windspeeds,
    measured_height=50, # The MERRA dataset offers windspeeds at 50m
    target_height=120,  # Assuming we want to project up to 120m
    roughness=np.array(roughness))

projected_windspeed.head()

,"(6.25000,51.00000)","(6.50000,51.00000)","(6.25000,50.75000)"
2015-01-01 00:30:00+00:00,7.280669,6.893002,6.839614
2015-01-01 01:30:00+00:00,7.316399,6.926829,6.873179
2015-01-01 02:30:00+00:00,7.261914,6.875246,6.821995
2015-01-01 03:30:00+00:00,7.027892,6.653684,6.602149
2015-01-01 04:30:00+00:00,7.129709,6.750080,6.697798
